In [1]:
import os
import re
import datetime
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import time as tm
pd.set_option('display.max_row', 1000)
%matplotlib inline
matplotlib.rcParams['savefig.dpi'] = 1.3 * matplotlib.rcParams['savefig.dpi']

In [2]:
pwd

u'/Users/Jonathan/Kaggle/Santander'

In [3]:
# Read data
df_train = pd.read_csv('data/train.csv').dropna(axis=0, how='all').reset_index(drop = True)
df_test = pd.read_csv('data/test.csv').dropna(axis=0, how='all').reset_index(drop = True)

In [4]:
from __future__ import division

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.svm import OneClassSVM

In [5]:
# remove constant columns
remove = []
for col in df_train.columns:
    if df_train[col].std() == 0:
        remove.append(col)

In [6]:
df_train.drop(remove, axis=1, inplace=True)
df_test.drop(remove, axis=1, inplace=True)

In [7]:
# remove duplicated columns
remove = []
c = df_train.columns
for i in range(len(c)-1):
    v = df_train[c[i]].values
    for j in range(i+1,len(c)):
        if np.array_equal(v,df_train[c[j]].values):
            remove.append(c[j])

In [8]:
df_train.drop(remove, axis=1, inplace=True)
df_test.drop(remove, axis=1, inplace=True)

y_train = df_train['TARGET'].values
X_train = df_train.drop(['ID','TARGET'], axis=1).values

In [9]:
id_test = df_test['ID']
X_test = df_test.drop(['ID'], axis=1).values

# length of dataset
len_train = len(X_train)
len_test  = len(X_test)

In [10]:
# 0.835920898605
# {'learning_rate': 0.125, 'max_depth': 3}

from sklearn.grid_search import GridSearchCV

t0 = tm.time()
depth = range(3,6)
ler_rate = np.linspace(0,0.5,5)
cs = np.linspace(0.5,1,5)

param_grid = dict(max_depth=depth, learning_rate = ler_rate, colsample_bytree=cs)
clf = xgb.XGBClassifier(nthread=4,subsample=0.95,seed=2016)
grid = GridSearchCV(clf, param_grid, cv=2, scoring='roc_auc')

grid.fit(X_train,y_train)
grid.grid_scores_

print 'time: ' +str((tm.time() - t0)/60) +' minutes'
# examine the best model
print grid.best_score_
print grid.best_params_
print grid.best_estimator_

time: 68.2539074699 minutes
0.836592540238
{'learning_rate': 0.125, 'colsample_bytree': 0.625, 'max_depth': 4}
XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.625,
       gamma=0, learning_rate=0.125, max_delta_step=0, max_depth=4,
       min_child_weight=1, missing=None, n_estimators=100, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=2016, silent=True, subsample=0.95)


In [ ]:
# # classifier
# clf = xgb.XGBClassifier(missing=np.nan, max_depth=5, n_estimators=X_train.shape[0], learning_rate=0.03, nthread=4, subsample=0.95, colsample_bytree=0.85, seed=4242)

# X_fit, X_eval, y_fit, y_eval= train_test_split(X_train, y_train, test_size=0.3)

# # fitting
# clf.fit(X_train, y_train, early_stopping_rounds=20, eval_metric="auc", eval_set=[(X_eval, y_eval)])

# print('Overall AUC:', roc_auc_score(y_train, clf.predict_proba(X_train)[:,1]))

In [12]:
# predicting
y_pred= grid.predict_proba(X_test)[:,1]

submission = pd.DataFrame({"ID":id_test, "TARGET":y_pred})
submission.to_csv("submission.csv", index=False)

print('Completed!')                

Completed!
